<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Tweepy 

In [ ]:
!pip install tweepy
!pip install google-cloud-bigquery

In [ ]:
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import bigquery


%matplotlib inline

In [ ]:
CONSUMER_KEY = 'CONSUMER_SECRET = ''
OAUTH_TOKEN = ''
OAUTH_TOKEN_SECRET = ''

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

api = tweepy.API(auth)

In [ ]:
import tweepy
import time
import json
#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        #print(status.extended_tweet.full_text)
        test=test

        
    def on_data(self, data):
        all_data = json.loads(data)
        if 'text' in all_data:
            tweetid=all_data["id"]
            tweet         = all_data["text"]
            created_at    = all_data["created_at"]
            retweeted     = all_data["retweeted"]
            userid      = all_data["user"]["id"]
            username      = all_data["user"]["screen_name"]
            userpage      = all_data["user"]["url"]
            user_followers       = all_data["user"]["followers_count"]
            user_location = all_data["user"]["location"]
            user_coordinates = all_data["coordinates"]
            user_friends = all_data["user"]["friends_count"]
            user_statuscount = all_data["user"]["statuses_count"]
            user_createdat = all_data["user"]["created_at"]
            quote_count=all_data["quote_count"]
            reply_count=all_data["reply_count"]
            retweet_count=all_data["retweet_count"]
            favourite_count=all_data["favorite_count"]
            
            retweetedText="none"
            retweetedStatus="none"
            tweetfull="none"
            if "retweeted_status" in all_data:
                if "extended_tweet" in all_data["retweeted_status"]:
                    retweetedText= all_data["retweeted_status"]["extended_tweet"]["full_text"]
                    retweetedStatus="true"
                    #print(retweetedText)

            
            if "extended_tweet" in all_data:
                tweetfull=all_data["extended_tweet"]["full_text"]
                #print(tweetfull)
                
            writeToGBQ(str(tweetid),str(created_at), str(retweeted), str(userid), str(username), str(userpage),\
                       str(user_followers), str(user_location), str(user_coordinates), str(user_friends), str(user_statuscount),\
                       str(user_createdat), str(quote_count), str(reply_count),\
                       str(retweet_count), str(favourite_count), str(retweetedText),str(retweetedStatus),str(tweetfull),\
                       str(tweet))
              
    def on_error(self, status_code):
        if status_code == 420:
            time.sleep(120)
            print("StatusCode420")
            #returning False in on_data disconnects the stream
            return False

In [2]:
def writeToGBQ(tweetid,created_at, retweeted, userid, username, userpage, user_followers, user_location, user_coordinates,\
               user_friends, user_statuscount, user_createdat, quote_count, reply_count, retweet_count,\
               favourite_count, retweetedText, retweetedStatus, tweetfull, tweet):
    #client = bigquery.Client()
    client = bigquery.Client.from_service_account_json('M11-Assignment-5b1de712e662.json')
    dataset_id = 'EuropawahlTweets'  # replace with your dataset ID
    #For this sample, the table must already exist and have a defined schema
    table_id = 'stream'  # replace with your table ID
    table_ref = client.dataset(dataset_id).table(table_id)
    table = client.get_table(table_ref)  # API request

    rows_to_insert = [
        (tweetid,created_at, retweeted, userid, username, userpage, user_followers, user_location, user_coordinates,\
               user_friends, user_statuscount, user_createdat, quote_count, reply_count, retweet_count,\
               favourite_count, retweetedText, retweetedStatus, tweetfull, tweet),
    ]

    errors = client.insert_rows(table, rows_to_insert)  # API request

    assert errors == []

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener, tweet_mode='extended')

In [ ]:
list_users = ['832097871893770240','743724264', '16705726', '4097559143', '828623998921629696', '20399735', '20463983', '710201540116652033']

In [ ]:
myStream.filter(track=['Europawahl'],follow=list_users, is_async=True)

In [ ]:
# TODO(developer): Uncomment the lines below and replace with your values.


